<a href="https://colab.research.google.com/github/Tarun-2109/C_F/blob/main/PySpark_Assignment_Feb25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CW1 - Amazon Bestsellers Analysis with PySpark


In this assignment you will be tasked with exploring a dataset containing the Top 50 best-selling books from Amazon between 2009-2019. You should complete the exercises presented in the Google Colab Notebook below. This assignment will be graded using CodeGrade.

Exercise 1 (5 Marks): Find the authors with the most entries in the bestseller’s lists, find the number of unique titles for each, the average rating, total number of reviews, and highest position in the ranking.

Exercise 2 (5 Marks): For fiction and non-fiction books, find the average and total number of reviews for the top 10, 25, and 50 of the bestsellers lists, in each year.

Exercise 3 (10 Marks): For each year, find the average price of a fiction and non-fiction book in the top 10, 25 and 50 of the bestsellers lists.

Exercise 4 (10 Marks): For free books—where the price is zero—fine the number of unique titles and authors. Compare the average rating and number of reviews in each year between free and priced books.


In [69]:
# CodeGrade Tag Init1

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [70]:
# Apache Spark uses Java, so first we must install that
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Unpack Spark from google drive
!tar xzf /content/drive/MyDrive/spark-3.3.0-bin-hadoop3.tgz

# Set up environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.3.0-bin-hadoop3"

# Install findspark, which helps python locate the psyspark module files
!pip install -q findspark
import findspark
findspark.init()
os.listdir('/content/drive/MyDrive/')

['AdmitCard_190310035793 (1).pdf',
 'Untitled document.gdoc',
 '20200413_235129.pdf',
 'AP chapter 2.pdf',
 'AP 1st chapter.pdf',
 'preview.jpg',
 'student_certificate.pdf',
 'R18 B.TECH EEE II Year Syllabus (1).pdf',
 'R18 B.TECH EEE II Year Syllabus.pdf',
 'IMG_0089.jpg',
 'IMG_0087 (1).jpg',
 'IMG_0083 (1).jpg',
 'IMG_0087.jpg',
 'IMG_0083.jpg',
 'TCS Hiring.pdf',
 'Freshers',
 'IMG_8114.JPG',
 'IMG_8126.JPG',
 'IMG_8118.JPG',
 'IMG_8129.JPG',
 'IMG_8117.JPG',
 'IMG_8116.JPG',
 'IMG_8113.JPG',
 'IMG_8127.JPG',
 'IMG_8124.JPG',
 'IMG_8120.JPG',
 'IMG_8128.JPG',
 'IMG_8119.JPG',
 'IMG_8115.JPG',
 'IMG_8130.JPG',
 'IMG_8121.JPG',
 'Untitled form.gform',
 'FRESHERS CAM-2',
 'chintu',
 'TARUN KUMAR (1).pdf',
 'TARUN KUMAR.pdf',
 'Allahabad to Trivandrum',
 'HR Salla Tarun Tarun.gsheet',
 'Tarun Resume - 2.pdf',
 'TARUN KUMAR RESUME 2 (1).pdf',
 'TARUN KUMAR RESUME 2.pdf',
 'TARUN KUMAR RESUME-2.pdf',
 'Team roster.gsheet',
 'Pros and cons.gsheet',
 'Schools in Dubai',
 '19261A0246_SALLA 

In [71]:
# Finally, we initialse a "SparkSession", which handles the computations
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

from pyspark.sql import functions as F

In [72]:
# Load the AmazonBooks.csv file into your notebook as a pyspark dataframe

CsvPath = '/content/drive/MyDrive/AmazonBooks-1.csv'

# Load .csv with header, ',' seperators and inferred schema
BooksDF = spark.read\
                     .option('header', 'True')\
                     .option('sep', ',')\
                     .option('inferSchema', 'True')\
                     .csv(CsvPath)

In [73]:
# CodeGrade Tag Init3

BooksDF.printSchema()
BooksDF.show()

root
 |-- Name: string (nullable = true)
 |-- Author: string (nullable = true)
 |-- User Rating: double (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Rank: integer (nullable = true)

+--------------------+----------------+-----------+-------+-----+----+-----------+----+
|                Name|          Author|User Rating|Reviews|Price|Year|      Genre|Rank|
+--------------------+----------------+-----------+-------+-----+----+-----------+----+
|            The Help|Kathryn Stockett|        4.8|  13871|    6|2009|    Fiction|   1|
|Where the Wild Th...|  Maurice Sendak|        4.8|   9967|   13|2009|    Fiction|   2|
|The Last Olympian...|    Rick Riordan|        4.8|   4628|    7|2009|    Fiction|   3|
|Diary of a Wimpy ...|     Jeff Kinney|        4.8|   3837|   15|2009|    Fiction|   4|
|            Watchmen|      Alan Moore|        4.8|   3829|   42|2009|  

In [74]:
# pyspark.sql.functions countains all the transformations and actions you will
# need
from pyspark.sql import functions as F

# Exercise 1

Find the authors with the most entries in the bestsellers lists. Find the number of unique titles for each author, the average rating, total number of reviews and highest position in the ranking. Create a dataframe where the columns are:

Author, Number of titles, Average Rating, Total Ratings, Highest Position

Sort by the number of titles in descending order.

In [75]:
# CodeGrade Tag Ex1
### Create a dataframe that contains, for each author, the number of unique
### books, the average rating, the number of reviews and the highest rank reached

Bestselling_Authors = BooksDF.groupBy("Author").agg(F.countDistinct("Name").alias("Number_of_Titles"),
                              F.avg("User Rating").alias("Average_Rating"),
                              F.sum("Reviews").alias("Total_Ratings"),
                              F.min("Rank").alias("Highest_Position")).orderBy(F.desc("Number_of_Titles"))

Bestselling_Authors = Bestselling_Authors.select(F.col("Author"),
                                     F.col("Number_of_Titles"),
                                     F.col("Average_Rating"),
                                     F.col("Total_Ratings"),
                                     F.col("Highest_Position"))

Bestselling_Authors.show()

+----------------+----------------+------------------+-------------+----------------+
|          Author|Number_of_Titles|    Average_Rating|Total_Ratings|Highest_Position|
+----------------+----------------+------------------+-------------+----------------+
|     Jeff Kinney|              13| 4.807692307692306|        93529|               4|
|    Rick Riordan|              10|4.7727272727272725|        44169|               3|
| Stephenie Meyer|               8|             4.675|       108273|              12|
|      Dav Pilkey|               7|               4.9|        82541|               4|
|   Bill O'Reilly|               6| 4.642857142857143|        63787|              13|
|    J.K. Rowling|               6|              4.45|        70535|               2|
| Suzanne Collins|               6| 4.666666666666667|       315502|               3|
|       E L James|               5| 4.233333333333333|       178011|              31|
|    John Grisham|               5|               4.4|

# Exercise 2

For fiction and non-fiction books, find the average rating, the average number of reviews, the total number of reviews and the average price in the bestsellers list, for each year. Create a dataframe where the columns are:

Year, Genre, Average Rating, Average Number of Reviews, Total Reviews, Average Price,

Sort by the year in ascending order.

In [76]:
# CodeGrade Tag Ex2
### Create a dataframe that shows the average user rating, average number of
### reviews, total number of reviews and average price in each year of the
### bestsellers list

Bestselling_Each_year = BooksDF.groupby("Year", "Genre").agg(F.avg("User Rating").alias("Average_Rating"),
                                                             F.avg("Reviews").alias("Average_Number_of_Reviews"),
                                                             F.sum("Reviews").alias("Total_Reviews"),
                                                             F.avg("Price").alias("Average_Price")).orderBy(F.asc("Year"))

Bestselling_Each_year = Bestselling_Each_year.select(F.col("Year"),
                                     F.col("Genre"),
                                     F.col("Average_Rating"),
                                     F.col("Average_Number_of_Reviews"),
                                     F.col("Total_Reviews"),
                                     F.col("Average_Price"))

Bestselling_Each_year.show()

+----+-----------+------------------+-------------------------+-------------+------------------+
|Year|      Genre|    Average_Rating|Average_Number_of_Reviews|Total_Reviews|     Average_Price|
+----+-----------+------------------+-------------------------+-------------+------------------+
|2009|    Fiction| 4.591666666666667|        6534.333333333333|       156824|15.583333333333334|
|2009|Non Fiction| 4.576923076923077|        3026.230769230769|        78682| 15.23076923076923|
|2010|    Fiction|             4.615|                  8409.25|       168185|               9.7|
|2010|Non Fiction|4.5200000000000005|       3526.5333333333333|       105796|              16.0|
|2011|    Fiction|4.6190476190476195|       10335.285714285714|       217041|11.619047619047619|
|2011|Non Fiction| 4.513793103448277|        6482.758620689655|       188000|17.620689655172413|
|2012|    Fiction| 4.495238095238096|       19896.238095238095|       417821|12.285714285714286|
|2012|Non Fiction| 4.558620689

# Exercise 3

For each year, find the average price of fiction and non-fiction books in the top 10, 25 and 50 of the bestsellers list. Make a dataframe where the columns are:

Year, Genre, Avg Price in Top 10, Avg Price in Top 25 and Avg Price in Top 50

Sort by the year in ascending order.

In [77]:
from typing_extensions import Final
# CodeGrade Tag Ex3
### Create a DataFrame that shows the average price for books in the top 10, 25
### and 50 of the bestsellers list, for each year in the dataset

BooksDF_sorted = BooksDF.orderBy(F.col("Reviews").desc())


Top10_DF = BooksDF_sorted.filter(F.col("Rank") <= 10)
Top25_DF = BooksDF_sorted.filter(F.col("Rank") <= 25)
Top50_DF = BooksDF_sorted.filter(F.col("Rank") <= 50)

Top10_Avg_price_DF = Top10_DF.groupby("Year", "Genre").agg(F.avg("Price").alias("Avg_Price_Top10"))
Top25_Avg_price_DF = Top25_DF.groupby("Year", "Genre").agg(F.avg("Price").alias("Avg_Price_Top25"))
Top50_Avg_price_DF = Top50_DF.groupby("Year", "Genre").agg(F.avg("Price").alias("Avg_Price_Top50"))

Final_DF = Top10_Avg_price_DF.join(Top25_Avg_price_DF, ["Year", "Genre"], "inner").join(Top50_Avg_price_DF, ["Year", "Genre"], "inner")

Final_DF = Final_DF.orderBy(F.asc("Year"))

Final_DF.show()

+----+-----------+------------------+------------------+------------------+
|Year|      Genre|   Avg_Price_Top10|   Avg_Price_Top25|   Avg_Price_Top50|
+----+-----------+------------------+------------------+------------------+
|2009|    Fiction|15.833333333333334|18.866666666666667|15.583333333333334|
|2009|Non Fiction|              14.5|              19.1| 15.23076923076923|
|2010|    Fiction|10.777777777777779|10.928571428571429|               9.7|
|2010|Non Fiction|              16.0|15.636363636363637|              16.0|
|2011|    Fiction|            10.375|10.923076923076923|11.619047619047619|
|2011|Non Fiction|              12.0|15.666666666666666|17.620689655172413|
|2012|    Fiction|13.333333333333334|              11.9|12.285714285714286|
|2012|Non Fiction|             17.25|17.933333333333334|17.482758620689655|
|2013|    Fiction| 9.333333333333334| 9.357142857142858|10.708333333333334|
|2013|Non Fiction|               8.0|14.363636363636363|18.192307692307693|
|2014|    Fi

# Exercise 4

For free books, find the total number of unique title and author, store these as variables called ```free_titles``` and ```free_authors```.

Compare the average rating and number of reviews for free and priced books, in each year of the dataset. Create a dataframe where the columns are:

Year, Avg Rating Free, Avg Rating Priced, Total Ratings Free, Total Ratings Priced

Sort by the year in ascending order.

In [78]:
# CodeGrade Tag Ex4a
### Find the number of free books in the dataset and the number of authors
### who wrote them

Free_Books_df = BooksDF.filter(F.col("Price") == 0)
Priced_Books_df = BooksDF.filter(F.col("Price") > 0)

Free_titles = Free_Books_df.select("Name").distinct().count()
Free_authors = Free_Books_df.select("Author").distinct().count()

print(f"Number of unique titles in free books: {Free_titles}")
print(f"Number of unique authors in free books: {Free_authors}")

Number of unique titles in free books: 9
Number of unique authors in free books: 6


In [79]:
# CodeGrade Tag Exb
### Create a dataframe that has, for each year, the average rating and number of
### user reviews for free books and priced books

Avg_Free_Books = Free_Books_df.groupby("Year").agg(F.avg("User Rating").alias("Avg_Rating_Free"),
                                                   F.sum("Reviews").alias("Total_Ratings_Free"))
Avg_Priced_Books = Priced_Books_df.groupby("Year").agg(F.avg("User Rating").alias("Avg_Rating_Priced"),
                                                       F.sum("Reviews").alias("Total_Ratings_Priced"))
Average_Final_DF = Avg_Free_Books.join(Avg_Priced_Books, ["Year"], "inner")
Average_Final_DF = Average_Final_DF.orderBy(F.asc("Year"))
Average_Final_DF.show()

+----+---------------+------------------+-----------------+--------------------+
|Year|Avg_Rating_Free|Total_Ratings_Free|Avg_Rating_Priced|Total_Ratings_Priced|
+----+---------------+------------------+-----------------+--------------------+
|2010|            4.6|              2122|4.557142857142857|              271859|
|2011|            4.8|              4505|4.553061224489797|              400536|
|2013|            4.8|             33046|          4.54375|              621861|
|2014|           4.75|             32738|4.610869565217391|              760259|
|2015|            4.8|             26234|4.644897959183671|              685435|
|2016|            4.8|             29008|4.672916666666665|              680792|
|2017|            4.8|              5836|4.657142857142857|              638584|
+----+---------------+------------------+-----------------+--------------------+

